In [ ]:
import pandas as pd
import numpy as np
import re
import string
import random
from tqdm.auto import tqdm
import torch
import transformers
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from transformers import AdamW, get_cosine_schedule_with_warmup
from transformers import pipeline, set_seed

In [ ]:
set_seed(42)

In [ ]:
df = pd.read_csv("/kaggle/input/summary-data/car_Summarized_Description.csv")

In [75]:
df.head(3)

,Unnamed: 0,car type,car name,year,url,Pros,Cons,Full Description,rating,current price Min,current price Max,base msrp,engine,drivetrain,dimensions,num_words,Full Name,GPT-2 Summarization,num_words_summarized
0,0,jeep,jeep wrangler,1998,https://www.edmunds.com/jeep/wrangler/1998/rev...,"['Unmatched off-road capability, overflowing w...","['Soft top is fun, but still a pain in the $%!...",Jeep has improved off-road capability by incre...,4.3 out of 5 stars,"$2,423","$4,074",N/A \n,"Inline 4 cylinder \n Horsepower: 120 hp @ 5,40...",Type: four wheel drive \n Transmission: 5-spee...,Length: 147.7 in. / Height: 69.6 in. \n Overal...,427,jeep wrangler 1998,Jeep has improved off-road capability by incre...,427
1,1,toyota,toyota 4runner,2005,https://www.edmunds.com/toyota/4runner/2005/re...,"['Powerful engine lineup, well mannered on pav...","[""Cargo capacity isn't much more than what man...",The base V6 now comes with a five-speed automa...,4.8 out of 5 stars,"$4,364","$6,983","$27,795 \n","V6 cylinder \n Horsepower: 245 hp @ 5,200 rpm ...",Type: rear wheel drive \n Transmission: 5-spee...,Length: 189.0 in. / Height: 68.5 in. \n Overal...,110,toyota 4runner 2005,The base V6 now comes with a five-speed automa...,110
2,2,toyota,toyota tundra,2002,https://www.edmunds.com/toyota/tundra/2002/rev...,"['Silky V8, Toyota build quality, less-than-fu...",['Lacks wide range of choice offered by domest...,"SR5 models have new 16-inch wheels, and a limi...",4.7 out of 5 stars,"$3,324","$5,612","$15,605 \n","V6 cylinder \n Horsepower: 190 hp @ 4,800 rpm ...",Type: rear wheel drive \n Transmission: 5-spee...,Length: 217.5 in. / Height: 70.5 in. \n Bed Le...,534,toyota tundra 2002,"SR5 models have new 16-inch wheels, and a limi...",534


In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3463 entries, 0 to 3462
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            3463 non-null   int64 
 1   car type              3463 non-null   object
 2   car name              3463 non-null   object
 3   year                  3463 non-null   int64 
 4   url                   3463 non-null   object
 5   Pros                  3463 non-null   object
 6   Cons                  3463 non-null   object
 7   Full Description      3463 non-null   object
 8   rating                3044 non-null   object
 9   current price Min     3065 non-null   object
 10  current price Max     3065 non-null   object
 11  base msrp             3410 non-null   object
 12  engine                3406 non-null   object
 13  drivetrain            3456 non-null   object
 14  dimensions            3448 non-null   object
 15  num_words             3463 non-null   

In [77]:
df.isna().sum()

Unnamed: 0                0
car type                  0
car name                  0
year                      0
url                       0
Pros                      0
Cons                      0
Full Description          0
rating                  419
current price Min       398
current price Max       398
base msrp                53
engine                   57
drivetrain                7
dimensions               15
num_words                 0
Full Name                 0
GPT-2 Summarization       0
num_words_summarized      0
dtype: int64

In [78]:
df_QAS=df.drop(columns=['Unnamed: 0','car type','url','current price Min','current price Max','num_words_summarized','num_words'])

In [79]:
df_QAS

,car name,year,Pros,Cons,Full Description,rating,base msrp,engine,drivetrain,dimensions,Full Name,GPT-2 Summarization
0,jeep wrangler,1998,"['Unmatched off-road capability, overflowing w...","['Soft top is fun, but still a pain in the $%!...",Jeep has improved off-road capability by incre...,4.3 out of 5 stars,N/A \n,"Inline 4 cylinder \n Horsepower: 120 hp @ 5,40...",Type: four wheel drive \n Transmission: 5-spee...,Length: 147.7 in. / Height: 69.6 in. \n Overal...,jeep wrangler 1998,Jeep has improved off-road capability by incre...
1,toyota 4runner,2005,"['Powerful engine lineup, well mannered on pav...","[""Cargo capacity isn't much more than what man...",The base V6 now comes with a five-speed automa...,4.8 out of 5 stars,"$27,795 \n","V6 cylinder \n Horsepower: 245 hp @ 5,200 rpm ...",Type: rear wheel drive \n Transmission: 5-spee...,Length: 189.0 in. / Height: 68.5 in. \n Overal...,toyota 4runner 2005,The base V6 now comes with a five-speed automa...
2,toyota tundra,2002,"['Silky V8, Toyota build quality, less-than-fu...",['Lacks wide range of choice offered by domest...,"SR5 models have new 16-inch wheels, and a limi...",4.7 out of 5 stars,"$15,605 \n","V6 cylinder \n Horsepower: 190 hp @ 4,800 rpm ...",Type: rear wheel drive \n Transmission: 5-spee...,Length: 217.5 in. / Height: 70.5 in. \n Bed Le...,toyota tundra 2002,"SR5 models have new 16-inch wheels, and a limi..."
3,toyota tacoma,1998,"['Unique styling, Toyota reliability, and powe...",['Extended cab models lack a third door and th...,The 1998 four-wheel-drive Tacomas receive fres...,4.6 out of 5 stars,N/A \n,"Inline 4 cylinder \n Horsepower: 142 hp @ 5,00...",Type: rear wheel drive \n Transmission: 5-spee...,Length: 203.1 in. / Height: 62.0 in. \n Overal...,toyota tacoma 1998,The 1998 four-wheel-drive Tacomas receive fres...
4,ford ranger,1990,empty,empty,No changes for the Ford Ranger.,4.1 out of 5 stars,N/A \n,"Inline 4 cylinder \n Horsepower: 100 hp @ 4,60...",Type: four wheel drive \n Transmission: 5-spee...,Length: 188.5 in. / Height: 63.6 in. \n Overal...,ford ranger 1990,No changes for the Ford Ranger.
...,...,...,...,...,...,...,...,...,...,...,...,...
3458,jeep renegade,2023,"[""Trailhawk model's excellent off-road capabil...",['High price when optioned with nice-to-have f...,The 2023 Jeep Renegade is a versatile subcomp...,NaN,"$31,590 \n","Inline 4 cylinder \n Horsepower: 177 hp @ 5,75...",Type: four wheel drive \n Transmission: 9-spee...,Length: 166.6 in. / Height: 66.5 in. \n Overal...,jeep renegade 2023,The 2023 Jeep Renegade is a versatile subcomp...
3459,bmw alpina-b7,2022,"['Smooth yet ridiculously quick acceleration',...","['Unintuitive gesture controls', ""Rear seats d...",High-performance M models have proliferated t...,NaN,"$144,195 \n","V8 cylinder \n Horsepower: 600 hp @ 5,500 rpm ...",Type: all wheel drive \n Transmission: 8-speed...,Length: 207.4 in. / Height: 58.7 in. \n Overal...,bmw alpina-b7 2022,High-performance M models have proliferated t...
3460,chevrolet colorado,2022,['Gasoline V6 and diesel four-cylinder engines...,['Front seats can feel confining for larger pe...,"After a handful of minor changes last year, t...",NaN,"$30,695 \n","Inline 4 cylinder \n Horsepower: 237 hp @ 5,60...",Type: rear wheel drive \n Transmission: 8-spee...,Length: 213.0 in. / Height: 78.8 in. \n Bed Le...,chevrolet colorado 2022,"After a handful of minor changes last year, t..."
3461,bmw 4-series,2023,"['Powerful and fuel-efficient engines', ""Sport...",['Flat hood makes it hard to place the front e...,The 4 Series is based on the 3 Series sedan b...,NaN,"$59,245 \n","Inline 6 cylinder \n Horsepower: 382 hp @ 5,80...",Type: rear wheel drive \n Transmission: 8-spee...,Length: 188.0 in. / Height: 54.6 in. \n Overal...,bmw 4-series 2023,The 4 Series is based on the 3 Series sedan b...


In [83]:
# extract horsepower values using a regular expression
df['horsepower'] = df['engine'].str.extract(r'Horsepower: (\d+) hp')

# show the resulting dataframe
print(df['horsepower'])

0       120
1       245
2       190
3       142
4       100
       ... 
3458    177
3459    600
3460    237
3461    382
3462    385
Name: horsepower, Length: 3463, dtype: object


In [84]:
df.head(3)

,Unnamed: 0,car type,car name,year,url,Pros,Cons,Full Description,rating,current price Min,...,base msrp,engine,drivetrain,dimensions,num_words,Full Name,GPT-2 Summarization,num_words_summarized,Horsepower,horsepower
0,0,jeep,jeep wrangler,1998,https://www.edmunds.com/jeep/wrangler/1998/rev...,"['Unmatched off-road capability, overflowing w...","['Soft top is fun, but still a pain in the $%!...",Jeep has improved off-road capability by incre...,4.3 out of 5 stars,"$2,423",...,N/A \n,"Inline 4 cylinder \n Horsepower: 120 hp @ 5,40...",Type: four wheel drive \n Transmission: 5-spee...,Length: 147.7 in. / Height: 69.6 in. \n Overal...,427,jeep wrangler 1998,Jeep has improved off-road capability by incre...,427,NaN,120
1,1,toyota,toyota 4runner,2005,https://www.edmunds.com/toyota/4runner/2005/re...,"['Powerful engine lineup, well mannered on pav...","[""Cargo capacity isn't much more than what man...",The base V6 now comes with a five-speed automa...,4.8 out of 5 stars,"$4,364",...,"$27,795 \n","V6 cylinder \n Horsepower: 245 hp @ 5,200 rpm ...",Type: rear wheel drive \n Transmission: 5-spee...,Length: 189.0 in. / Height: 68.5 in. \n Overal...,110,toyota 4runner 2005,The base V6 now comes with a five-speed automa...,110,NaN,245
2,2,toyota,toyota tundra,2002,https://www.edmunds.com/toyota/tundra/2002/rev...,"['Silky V8, Toyota build quality, less-than-fu...",['Lacks wide range of choice offered by domest...,"SR5 models have new 16-inch wheels, and a limi...",4.7 out of 5 stars,"$3,324",...,"$15,605 \n","V6 cylinder \n Horsepower: 190 hp @ 4,800 rpm ...",Type: rear wheel drive \n Transmission: 5-spee...,Length: 217.5 in. / Height: 70.5 in. \n Bed Le...,534,toyota tundra 2002,"SR5 models have new 16-inch wheels, and a limi...",534,245.0,190


In [85]:
# extract engine type using a regular expression
df['engine_type'] = df['engine'].str.extract(r'([A-Za-z0-9\s]+)cylinder')

In [86]:
df.isna().sum()

Unnamed: 0                 0
car type                   0
car name                   0
year                       0
url                        0
Pros                       0
Cons                       0
Full Description           0
rating                   419
current price Min        398
current price Max        398
base msrp                 53
engine                    57
drivetrain                 7
dimensions                15
num_words                  0
Full Name                  0
GPT-2 Summarization        0
num_words_summarized       0
Horsepower              1629
horsepower                59
engine_type               68
dtype: int64

In [87]:
nan_engine_type = df[pd.isna(df['engine_type'])]

In [93]:
nan_engine_type[pd.isna(nan_engine_type['engine'])==False]

,Unnamed: 0,car type,car name,year,url,Pros,Cons,Full Description,rating,current price Min,...,engine,drivetrain,dimensions,num_words,Full Name,GPT-2 Summarization,num_words_summarized,Horsepower,horsepower,engine_type
551,551,hyundai,hyundai nexo,2021,https://www.edmunds.com/hyundai/nexo/2021/review/,['Promises 350-plus miles of range from a five...,['Only available in select areas of California...,Automakers are increasingly bullish on electr...,5.0 out of 5 stars,"$27,133",...,Horsepower: 161 hp \n Torque: 291 lb-ft \n,Type: front wheel drive \n Transmission: 1-spe...,Length: 183.9 in. / Height: 64.2 in. \n Overal...,380,hyundai nexo 2021,Automakers are increasingly bullish on electr...,380,NaN,161,NaN
570,570,hyundai,hyundai nexo,2019,https://www.edmunds.com/hyundai/nexo/2019/review/,['Promises 350-plus miles of range from a five...,['Only available in select areas of California...,"Alternatively fueled vehicles are the future,...",5.0 out of 5 stars,"$19,751",...,Horsepower: 161 hp \n Torque: 291 lb-ft \n,Type: front wheel drive \n Transmission: 1-spe...,Length: 183.9 in. / Height: 64.2 in. \n Overal...,381,hyundai nexo 2019,"Alternatively fueled vehicles are the future,...",381,NaN,161,NaN
599,599,hyundai,hyundai nexo,2020,https://www.edmunds.com/hyundai/nexo/2020/review/,['Promises 350-plus miles of range from a five...,['Only available in select areas of California...,While automakers are generally betting big on...,5.0 out of 5 stars,"$25,182",...,Horsepower: 161 hp \n Torque: 291 lb-ft \n,Type: front wheel drive \n Transmission: 1-spe...,Length: 183.9 in. / Height: 64.2 in. \n Overal...,319,hyundai nexo 2020,While automakers are generally betting big on...,319,NaN,161,NaN
919,919,hyundai,hyundai nexo,2022,https://www.edmunds.com/hyundai/nexo/,['Promises 350-plus miles of range from a five...,['Only available in select areas of California...,"Each year, more and more battery electric car...",NaN,NaN,...,Horsepower: 161 hp \n Torque: 291 lb-ft \n,Type: front wheel drive \n Transmission: 1-spe...,Length: 183.9 in. / Height: 64.2 in. \n Overal...,318,hyundai nexo 2022,"Each year, more and more battery electric car...",318,161.0,161,NaN
979,979,toyota,toyota mirai,2023,https://www.edmunds.com/toyota/mirai/,['Longer driving range than many battery elect...,"['Hydrogen stations are few and far between', ...",People looking for alternatives to gas-powere...,NaN,NaN,...,Horsepower: 182 hp \n Torque: 221 lb-ft \n,Type: rear wheel drive \n Transmission: Contin...,Length: 195.8 in. / Height: 57.9 in. \n Overal...,340,toyota mirai 2023,People looking for alternatives to gas-powere...,340,NaN,182,NaN
1294,313,toyota,toyota mirai,2016,https://www.edmunds.com/toyota/mirai/2016/review/,"['Futuristic powertrain technology', 'snappy l...","['Pricey', 'extremely limited refueling locati...",The 2016 Toyota Mirai is an all-new model. The...,1.0 out of 5 stars,"$10,566",...,Horsepower: 151 hp \n Torque: 247 lb-ft \n,Type: front wheel drive \n Transmission: 1-spe...,Length: 192.5 in. / Height: 60.4 in. \n Overal...,1277,toyota mirai 2016,The 2016 Toyota Mirai is an all-new model. The...,219,151.0,151,NaN
1491,510,toyota,toyota mirai,2019,https://www.edmunds.com/toyota/mirai/2019/review/,['Long driving range bests the range of most b...,"['Hydrogen stations are few and far between', ...",For commuters who live in a region where the ...,2.9 out of 5 stars,"$17,484",...,Horsepower: 151 hp \n Torque: 247 lb-ft \n,Type: front wheel drive \n Transmission: 1-spe...,Length: 192.5 in. / Height: 60.5 in. \n Overal...,1063,toyota mirai 2019,For commuters who live in a region where the ...,211,151.0,151,NaN
1575,594,toyota,toyota mirai,2017,https://www.edmunds.com/toyota/mirai/2017/review/,"['Long driving range bests all other EVs', 'Co...","['Hydrogen stations are few and far between', ...","For 2017, the Toyota Mirai is unchanged. If al...",2.8 out of 5 stars,"$12,519",...,Horsepower: 151 hp \n Torque: 247 lb-ft \n,Type: front wheel drive \n Transmi

In [94]:
nan_horsepower = df[pd.isna(df['horsepower'])]

In [96]:
nan_horsepower.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59 entries, 546 to 3451
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            59 non-null     int64  
 1   car type              59 non-null     object 
 2   car name              59 non-null     object 
 3   year                  59 non-null     int64  
 4   url                   59 non-null     object 
 5   Pros                  59 non-null     object 
 6   Cons                  59 non-null     object 
 7   Full Description      59 non-null     object 
 8   rating                25 non-null     object 
 9   current price Min     25 non-null     object 
 10  current price Max     25 non-null     object 
 11  base msrp             25 non-null     object 
 12  engine                2 non-null      object 
 13  drivetrain            52 non-null     object 
 14  dimensions            52 non-null     object 
 15  num_words            

In [98]:
# extract torque using a regular expression
torque_regex = r'Torque: ([\d]+ [\w]+-[\w]+) @ ([\d,]+) rpm'
torque_matches = df['engine'].str.extract(torque_regex)
# check if any rows did not match the regular expression
if torque_matches.isna().any().any():
    print('Warning: Some rows did not match the regular expression for torque values.')
# add torque values to dataframe
if torque_matches.shape[1] == 2:
    df[['Torque', 'Torque_RPM']] = torque_matches
else:
    print('Error: The regular expression for torque values returned the wrong number of matches.')

In [99]:
nan_torque = df[pd.isna(df['Torque'])]

In [109]:
nan_torque.isna().sum()

Unnamed: 0                0
car type                  0
car name                  0
year                      0
url                       0
Pros                      0
Cons                      0
Full Description          0
rating                  114
current price Min       103
current price Max       103
base msrp                51
engine                   57
drivetrain                7
dimensions                9
num_words                 0
Full Name                 0
GPT-2 Summarization       0
num_words_summarized      0
Horsepower              161
horsepower               59
engine_type              68
Torque                  293
Torque_RPM              293
dtype: int64

In [108]:
nan_torque.loc[nan_torque['Torque'].isna(), 'engine'].iloc[0]

'V6 cylinder \n Horsepower: 354 hp @ 6,600 rpm \n '

In [110]:
df = df.dropna(subset=['Torque'])

In [113]:
df.isna().sum()

car name                  0
year                      0
Pros                      0
Cons                      0
Full Description          0
rating                  305
base msrp                 2
engine                    0
drivetrain                0
dimensions                6
Full Name                 0
GPT-2 Summarization       0
Horsepower             1468
horsepower                0
engine_type               0
Torque                    0
Torque_RPM                0
dtype: int64

In [ ]:
df=df.drop(columns=['Unnamed: 0','car type','url','current price Min','current price Max','num_words_summarized','num_words'])

In [116]:
df=df.drop(columns=['Horsepower'])

In [128]:
df.isna().sum()

car name                 0
year                     0
Pros                     0
Cons                     0
Full Description         0
rating                 305
base msrp                2
engine                   0
drivetrain               0
dimensions               6
Full Name                0
GPT-2 Summarization      0
horsepower               0
engine_type              0
Torque                   0
Torque_RPM               0
drive_type               0
transmission            17
dtype: int64

In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3170 entries, 0 to 3462
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   car name             3170 non-null   object
 1   year                 3170 non-null   int64 
 2   Pros                 3170 non-null   object
 3   Cons                 3170 non-null   object
 4   Full Description     3170 non-null   object
 5   rating               2865 non-null   object
 6   base msrp            3168 non-null   object
 7   engine               3170 non-null   object
 8   drivetrain           3170 non-null   object
 9   dimensions           3164 non-null   object
 10  Full Name            3170 non-null   object
 11  GPT-2 Summarization  3170 non-null   object
 12  horsepower           3170 non-null   object
 13  engine_type          3170 non-null   object
 14  Torque               3170 non-null   object
 15  Torque_RPM           3170 non-null   object
dtypes: int

In [120]:
df.loc[1, 'drivetrain']

'Type: rear wheel drive \n Transmission: 5-speed automatic \n '

In [122]:
df.loc[25, 'drivetrain']

'Type: four wheel drive \n Transmission: 5-speed manual \n '

In [123]:
df.loc[3000, 'drivetrain']

'Type: front wheel drive \n Transmission: Continuously variable-speed automatic \n '

In [126]:
transmission_regex = r'Transmission: ([\d\w\s\-]+)'
df['transmission'] = df['drivetrain'].str.extract(transmission_regex)

In [135]:
nan_t = df[pd.isna(df['transmission'])]

In [133]:
df.loc[pd.isna(df['transmission']), 'transmission'] = '5-speed manual'

In [136]:
nan_t

,car name,year,Pros,Cons,Full Description,rating,base msrp,engine,drivetrain,dimensions,Full Name,GPT-2 Summarization,horsepower,engine_type,Torque,Torque_RPM,drive_type,transmission


In [137]:
df.head(2)

,car name,year,Pros,Cons,Full Description,rating,base msrp,engine,drivetrain,dimensions,Full Name,GPT-2 Summarization,horsepower,engine_type,Torque,Torque_RPM,drive_type,transmission
0,jeep wrangler,1998,"['Unmatched off-road capability, overflowing w...","['Soft top is fun, but still a pain in the $%!...",Jeep has improved off-road capability by incre...,4.3 out of 5 stars,N/A \n,"Inline 4 cylinder \n Horsepower: 120 hp @ 5,40...",Type: four wheel drive \n Transmission: 5-spee...,Length: 147.7 in. / Height: 69.6 in. \n Overal...,jeep wrangler 1998,Jeep has improved off-road capability by incre...,120,Inline 4,140 lb-ft,"3,500",four wheel drive \n Transmission,5-speed manual \n
1,toyota 4runner,2005,"['Powerful engine lineup, well mannered on pav...","[""Cargo capacity isn't much more than what man...",The base V6 now comes with a five-speed automa...,4.8 out of 5 stars,"$27,795 \n","V6 cylinder \n Horsepower: 245 hp @ 5,200 rpm ...",Type: rear wheel drive \n Transmission: 5-spee...,Length: 189.0 in. / Height: 68.5 in. \n Overal...,toyota 4runner 2005,The base V6 now comes with a five-speed automa...,245,V6,282 lb-ft,"3,800",rear wheel drive \n Transmission,5-speed automatic \n


In [138]:
df.loc[25,'dimensions']

'Length: 188.3 in. / Height: 71.0 in. \n Overall Width without Mirrors: 76.4 in. \n '

In [139]:
df.loc[100,'dimensions']

'Length: 172.6 in. / Height: 55.7 in. \n Overall Width without Mirrors: 66.1 in. \n Curb Weight: 2,375 lbs. \n Cargo Capacity, All Seats In Place: 16.6 cu.ft. \n '

In [140]:
df.loc[150,'dimensions']

'Length: 190.2 in. / Height: 55.9 in. \n Overall Width without Mirrors: 70.3 in. \n Curb Weight: 3,245 lbs. \n Cargo Capacity, All Seats In Place: 15.4 cu.ft. \n '

In [141]:
df.loc[3000,'dimensions']

'Length: 175.6 in. / Height: 58.7 in. \n Overall Width without Mirrors: 68.7 in. \n Curb Weight: 3,042 lbs. \n Cargo Capacity, All Seats In Place: 21.6 cu.ft. \n '

In [142]:
df['length'] = df['dimensions'].str.extract(r'Length: (\d+\.\d+)').astype(float)
df['height'] = df['dimensions'].str.extract(r'Height: (\d+\.\d+)').astype(float)
df['width'] = df['dimensions'].str.extract(r'Overall Width without Mirrors: (\d+\.\d+|\d+\.\d+ \(.+\))').astype(str)

In [143]:
df.isna().sum()

car name                 0
year                     0
Pros                     0
Cons                     0
Full Description         0
rating                 305
base msrp                2
engine                   0
drivetrain               0
dimensions               6
Full Name                0
GPT-2 Summarization      0
horsepower               0
engine_type              0
Torque                   0
Torque_RPM               0
drive_type               0
transmission             0
length                  44
height                  47
width                    0
dtype: int64

In [167]:
nan_dim = df[pd.isna(df['Length'])]

In [169]:
nan_dim

,car name,year,Pros,Cons,Full Description,rating,base msrp,engine,drivetrain,dimensions,...,horsepower,engine_type,Torque,Torque_RPM,drive_type,transmission,length,height,width,Length
74,ford gt,2017,"['Max-attack handling capabilities', 'Race-bre...","[""Turbocharged V6 isn't brimming with personal...","If nothing else, you have to give credit to F...",NaN,"$447,000 \n","V6 cylinder \n Horsepower: 647 hp @ 6,250 rpm ...",Type: rear wheel drive \n Transmission: 7-spee...,NaN,...,647,V6,550 lb-ft,"5,900",rear wheel drive \n Transmission,7-speed automated manual \n,NaN,NaN,nan,NaN
326,audi rs-3,2017,['Ridiculous power in a relatively compact car...,"['Limited interior storage and cargo space', '...",The Audi RS 3 has been available in other mark...,5.0 out of 5 stars,"$54,500 \n","Inline 5 cylinder \n Horsepower: 400 hp @ 7,00...",Type: all wheel drive \n Transmission: 7-speed...,NaN,...,400,Inline 5,354 lb-ft,"1,700",all wheel drive \n Transmission,7-speed automated manual \n,NaN,NaN,nan,NaN
1421,audi tt-rs,2018,['An abundance of power from a turbo five-cyli...,['Tiny rear seats are better suited for cargo ...,"After about a five-year absence, the Audi TT R...",4.0 out of 5 stars,"$64,900 \n","Inline 5 cylinder \n Horsepower: 400 hp @ 7,00...",Type: all wheel drive \n Transmission: 7-speed...,NaN,...,400,Inline 5,354 lb-ft,"1,700",all wheel drive \n Transmission,7-speed automated manual \n,NaN,NaN,nan,NaN
2038,audi rs-3,2018,"['Potent acceleration and heaps of traction', ...","['Snug back seat and small cargo area', 'Ride ...","For Audi nerds, the arrival of the RS 3 on U....",4.9 out of 5 stars,"$54,900 \n","Inline 5 cylinder \n Horsepower: 400 hp @ 7,00...",Type: all wheel drive \n Transmission: 7-speed...,NaN,...,400,Inline 5,354 lb-ft,"1,700",all wheel drive \n Transmission,7-speed automated manual \n,NaN,NaN,nan,NaN
2488,audi s3,2017,"['Powerful turbocharged four-cylinder engine',...","[""Trunk won't fit much more than a couple smal...","For 2017, the Audi S3 receives a modest cosmet...",4.8 out of 5 stars,"$42,900 \n","Inline 4 cylinder \n Horsepower: 292 hp @ 5,40...",Type: all wheel drive \n Transmission: 6-speed...,"Curb Weight: 3,462 lbs. \n Cargo Capacity, All...",...,292,Inline 4,280 lb-ft,"1,900",all wheel drive \n Transmission,6-speed automated manual \n,NaN,NaN,nan,NaN
2624,audi a5,2017,['Classic styling that has stood the test of t...,['Certain high-tech features are not offered o...,"After nine years, this first-generation Audi A...",4.6 out of 5 stars,"$41,200 \n","Inline 4 cylinder \n Horsepower: 220 hp @ 4,45...",Type: all wheel drive \n Transmission: 6-speed...,NaN,...,220,Inline 4,258 lb-ft,"1,500",all wheel drive \n Transmission,6-speed manual \n,NaN,NaN,nan,NaN
2781,audi a5,2018,"[""Interior that's understated and classy"", 'St...",['Interior storage is limited'],"For 2018, the Audi A5 is fully redesigned. The...",4.6 out of 5 stars,"$49,600 \n","Inline 4 cylinder \n Horsepower: 252 hp @ 5,00...",Type: all wheel drive \n Transmission: 7-speed...,NaN,...,252,Inline 4,273 lb-ft,"1,600",all wheel drive \n Transmission,7-speed automated manual \n,NaN,NaN,nan,NaN


In [170]:
nan_dim.loc[2781,'dimensions']

nan

In [159]:
df.head(3)

,car name,year,Pros,Cons,Full Description,rating,base msrp,engine,drivetrain,dimensions,...,GPT-2 Summarization,horsepower,engine_type,Torque,Torque_RPM,drive_type,transmission,length,height,width
0,jeep wrangler,1998,"['Unmatched off-road capability, overflowing w...","['Soft top is fun, but still a pain in the $%!...",Jeep has improved off-road capability by incre...,4.3 out of 5 stars,N/A \n,"Inline 4 cylinder \n Horsepower: 120 hp @ 5,40...",Type: four wheel drive \n Transmission: 5-spee...,Length: 147.7 in. / Height: 69.6 in. \n Overal...,...,Jeep has improved off-road capability by incre...,120,Inline 4,140 lb-ft,"3,500",four wheel drive \n Transmission,5-speed manual \n,147.7,69.6,66.7
1,toyota 4runner,2005,"['Powerful engine lineup, well mannered on pav...","[""Cargo capacity isn't much more than what man...",The base V6 now comes with a five-speed automa...,4.8 out of 5 stars,"$27,795 \n","V6 cylinder \n Horsepower: 245 hp @ 5,200 rpm ...",Type: rear wheel drive \n Transmission: 5-spee...,Length: 189.0 in. / Height: 68.5 in. \n Overal...,...,The base V6 now comes with a five-speed automa...,245,V6,282 lb-ft,"3,800",rear wheel drive \n Transmission,5-speed automatic \n,189.0,68.5,73.8
2,toyota tundra,2002,"['Silky V8, Toyota build quality, less-than-fu...",['Lacks wide range of choice offered by domest...,"SR5 models have new 16-inch wheels, and a limi...",4.7 out of 5 stars,"$15,605 \n","V6 cylinder \n Horsepower: 190 hp @ 4,800 rpm ...",Type: rear wheel drive \n Transmission: 5-spee...,Length: 217.5 in. / Height: 70.5 in. \n Bed Le...,...,"SR5 models have new 16-inch wheels, and a limi...",190,V6,220 lb-ft,"3,600",rear wheel drive \n Transmission,5-speed manual \n,217.5,70.5,75.2


In [161]:
df['Length'] = df['dimensions'].str.split('Length: ').str[1].str.split(' in.').str[0] + ' in.'

In [162]:
df.head(3)

,car name,year,Pros,Cons,Full Description,rating,base msrp,engine,drivetrain,dimensions,...,horsepower,engine_type,Torque,Torque_RPM,drive_type,transmission,length,height,width,Length
0,jeep wrangler,1998,"['Unmatched off-road capability, overflowing w...","['Soft top is fun, but still a pain in the $%!...",Jeep has improved off-road capability by incre...,4.3 out of 5 stars,N/A \n,"Inline 4 cylinder \n Horsepower: 120 hp @ 5,40...",Type: four wheel drive \n Transmission: 5-spee...,Length: 147.7 in. / Height: 69.6 in. \n Overal...,...,120,Inline 4,140 lb-ft,"3,500",four wheel drive \n Transmission,5-speed manual \n,147.7,69.6,66.7,147.7 in.
1,toyota 4runner,2005,"['Powerful engine lineup, well mannered on pav...","[""Cargo capacity isn't much more than what man...",The base V6 now comes with a five-speed automa...,4.8 out of 5 stars,"$27,795 \n","V6 cylinder \n Horsepower: 245 hp @ 5,200 rpm ...",Type: rear wheel drive \n Transmission: 5-spee...,Length: 189.0 in. / Height: 68.5 in. \n Overal...,...,245,V6,282 lb-ft,"3,800",rear wheel drive \n Transmission,5-speed automatic \n,189.0,68.5,73.8,189.0 in.
2,toyota tundra,2002,"['Silky V8, Toyota build quality, less-than-fu...",['Lacks wide range of choice offered by domest...,"SR5 models have new 16-inch wheels, and a limi...",4.7 out of 5 stars,"$15,605 \n","V6 cylinder \n Horsepower: 190 hp @ 4,800 rpm ...",Type: rear wheel drive \n Transmission: 5-spee...,Length: 217.5 in. / Height: 70.5 in. \n Bed Le...,...,190,V6,220 lb-ft,"3,600",rear wheel drive \n Transmission,5-speed manual \n,217.5,70.5,75.2,217.5 in.


In [163]:
df.isna().sum()

car name                 0
year                     0
Pros                     0
Cons                     0
Full Description         0
rating                 305
base msrp                2
engine                   0
drivetrain               0
dimensions               6
Full Name                0
GPT-2 Summarization      0
horsepower               0
engine_type              0
Torque                   0
Torque_RPM               0
drive_type               0
transmission             0
length                  44
height                  47
width                    0
Length                   7
dtype: int64

In [182]:
nan_height = df[pd.isna(df['Height'])]

In [187]:
nan_height.loc[533,'dimensions']

'Length: 176.5 in. \n Overall Width without Mirrors: 66.8 in. \n '

In [189]:
df['Height'] = df['dimensions'].str.split('Height: ').str[1].str.split(' in.').str[0] + ' in.'

In [190]:
df.isna().sum()

car name                 0
year                     0
Pros                     0
Cons                     0
Full Description         0
rating                 305
base msrp                2
engine                   0
drivetrain               0
dimensions               6
Full Name                0
GPT-2 Summarization      0
horsepower               0
engine_type              0
Torque                   0
Torque_RPM               0
drive_type               0
transmission             0
length                  44
height                  47
width                    0
Length                   7
Height                  27
dtype: int64

In [191]:
df_QAS=df.drop(columns=['rating','length','height'])

In [192]:
df_QAS.loc[1,'base msrp']

'$27,795 \n '

In [194]:
df_QAS.isna().sum()

car name                0
year                    0
Pros                    0
Cons                    0
Full Description        0
base msrp               2
engine                  0
drivetrain              0
dimensions              6
Full Name               0
GPT-2 Summarization     0
horsepower              0
engine_type             0
Torque                  0
Torque_RPM              0
drive_type              0
transmission            0
width                   0
Length                  7
Height                 27
dtype: int64

In [195]:
df_QAS = df_QAS.dropna()

In [196]:
df_QAS.isna().sum()

car name               0
year                   0
Pros                   0
Cons                   0
Full Description       0
base msrp              0
engine                 0
drivetrain             0
dimensions             0
Full Name              0
GPT-2 Summarization    0
horsepower             0
engine_type            0
Torque                 0
Torque_RPM             0
drive_type             0
transmission           0
width                  0
Length                 0
Height                 0
dtype: int64

In [197]:
df_QAS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3141 entries, 0 to 3462
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   car name             3141 non-null   object
 1   year                 3141 non-null   int64 
 2   Pros                 3141 non-null   object
 3   Cons                 3141 non-null   object
 4   Full Description     3141 non-null   object
 5   base msrp            3141 non-null   object
 6   engine               3141 non-null   object
 7   drivetrain           3141 non-null   object
 8   dimensions           3141 non-null   object
 9   Full Name            3141 non-null   object
 10  GPT-2 Summarization  3141 non-null   object
 11  horsepower           3141 non-null   object
 12  engine_type          3141 non-null   object
 13  Torque               3141 non-null   object
 14  Torque_RPM           3141 non-null   object
 15  drive_type           3141 non-null   object
 16  transm

In [208]:
df_QAS.head(5)

,car name,year,Pros,Cons,Full Description,base msrp,engine,drivetrain,dimensions,Full Name,GPT-2 Summarization,horsepower,engine_type,Torque,Torque_RPM,drive_type,transmission,width,Length,Height
0,jeep wrangler,1998,"['Unmatched off-road capability, overflowing w...","['Soft top is fun, but still a pain in the $%!...",Jeep has improved off-road capability by incre...,N/A \n,"Inline 4 cylinder \n Horsepower: 120 hp @ 5,40...",Type: four wheel drive \n Transmission: 5-spee...,Length: 147.7 in. / Height: 69.6 in. \n Overal...,jeep wrangler 1998,Jeep has improved off-road capability by incre...,120,Inline 4,140 lb-ft,"3,500",four wheel drive \n,5-speed manual \n,66.7,147.7 in.,69.6 in.
1,toyota 4runner,2005,"['Powerful engine lineup, well mannered on pav...","[""Cargo capacity isn't much more than what man...",The base V6 now comes with a five-speed automa...,"$27,795 \n","V6 cylinder \n Horsepower: 245 hp @ 5,200 rpm ...",Type: rear wheel drive \n Transmission: 5-spee...,Length: 189.0 in. / Height: 68.5 in. \n Overal...,toyota 4runner 2005,The base V6 now comes with a five-speed automa...,245,V6,282 lb-ft,"3,800",rear wheel drive \n,5-speed automatic \n,73.8,189.0 in.,68.5 in.
2,toyota tundra,2002,"['Silky V8, Toyota build quality, less-than-fu...",['Lacks wide range of choice offered by domest...,"SR5 models have new 16-inch wheels, and a limi...","$15,605 \n","V6 cylinder \n Horsepower: 190 hp @ 4,800 rpm ...",Type: rear wheel drive \n Transmission: 5-spee...,Length: 217.5 in. / Height: 70.5 in. \n Bed Le...,toyota tundra 2002,"SR5 models have new 16-inch wheels, and a limi...",190,V6,220 lb-ft,"3,600",rear wheel drive \n,5-speed manual \n,75.2,217.5 in.,70.5 in.
3,toyota tacoma,1998,"['Unique styling, Toyota reliability, and powe...",['Extended cab models lack a third door and th...,The 1998 four-wheel-drive Tacomas receive fres...,N/A \n,"Inline 4 cylinder \n Horsepower: 142 hp @ 5,00...",Type: rear wheel drive \n Transmission: 5-spee...,Length: 203.1 in. / Height: 62.0 in. \n Overal...,toyota tacoma 1998,The 1998 four-wheel-drive Tacomas receive fres...,142,Inline 4,160 lb-ft,"4,000",rear wheel drive \n,5-speed manual \n,66.5,203.1 in.,62.0 in.
4,ford ranger,1990,empty,empty,No changes for the Ford Ranger.,N/A \n,"Inline 4 cylinder \n Horsepower: 100 hp @ 4,60...",Type: four wheel drive \n Transmission: 5-spee...,Length: 188.5 in. / Height: 63.6 in. \n Overal...,ford ranger 1990,No changes for the Ford Ranger.,100,Inline 4,133 lb-ft,"2,600",four wheel drive \n,5-speed manual \n,66.8,188.5 in.,63.6 in.


In [207]:
df_QAS['drive_type'] = df_QAS['drive_type'].str.replace('Transmission', '')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [211]:
cols_to_clean = ['base msrp', 'drive_type', 'transmission']
df_QAS[cols_to_clean] = df[cols_to_clean].applymap(lambda x: x.replace('\n', '') if isinstance(x, str) else x)


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [212]:
df_QAS.head(2)

,car name,year,Pros,Cons,Full Description,base msrp,engine,drivetrain,dimensions,Full Name,GPT-2 Summarization,horsepower,engine_type,Torque,Torque_RPM,drive_type,transmission,width,Length,Height
0,jeep wrangler,1998,"['Unmatched off-road capability, overflowing w...","['Soft top is fun, but still a pain in the $%!...",Jeep has improved off-road capability by incre...,N/A,"Inline 4 cylinder \n Horsepower: 120 hp @ 5,40...",Type: four wheel drive \n Transmission: 5-spee...,Length: 147.7 in. / Height: 69.6 in. \n Overal...,jeep wrangler 1998,Jeep has improved off-road capability by incre...,120,Inline 4,140 lb-ft,"3,500",four wheel drive,5-speed manual,66.7,147.7 in.,69.6 in.
1,toyota 4runner,2005,"['Powerful engine lineup, well mannered on pav...","[""Cargo capacity isn't much more than what man...",The base V6 now comes with a five-speed automa...,"$27,795","V6 cylinder \n Horsepower: 245 hp @ 5,200 rpm ...",Type: rear wheel drive \n Transmission: 5-spee...,Length: 189.0 in. / Height: 68.5 in. \n Overal...,toyota 4runner 2005,The base V6 now comes with a five-speed automa...,245,V6,282 lb-ft,"3,800",rear wheel drive,5-speed automatic,73.8,189.0 in.,68.5 in.


In [213]:
df_QAS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3141 entries, 0 to 3462
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   car name             3141 non-null   object
 1   year                 3141 non-null   int64 
 2   Pros                 3141 non-null   object
 3   Cons                 3141 non-null   object
 4   Full Description     3141 non-null   object
 5   base msrp            3141 non-null   object
 6   engine               3141 non-null   object
 7   drivetrain           3141 non-null   object
 8   dimensions           3141 non-null   object
 9   Full Name            3141 non-null   object
 10  GPT-2 Summarization  3141 non-null   object
 11  horsepower           3141 non-null   object
 12  engine_type          3141 non-null   object
 13  Torque               3141 non-null   object
 14  Torque_RPM           3141 non-null   object
 15  drive_type           3141 non-null   object
 16  transm

In [215]:
df_QAS.to_csv('/kaggle/working/Summary_data_all_columns.csv', index=False)

In [1]:
def generate_question(row):
    car_name = row['Full Name']
    question = random.choice([
        f"What is the horsepower rating of the engine in the {car_name}?",
        f"How much horsepower does the {car_name}'s engine produce?",
        f"What is the horsepower output of the {car_name}'s engine?",
        f"Can you tell me the horsepower of the engine in the {car_name}?",
        f"How many horsepower does the {car_name}'s engine generate?"
    ])
    return question

In [2]:
QAS = pd.DataFrame(columns=['question', 'answer', 'start_pos', 'end_pos', 'final_context'])

NameError: name 'pd' is not defined